In [9]:
# # Install environment and agent
#!pip install highway-env
# # TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
#!pip install git+https://github.com/DLR-RM/stable-baselines3

# # Environment
import gymnasium as gym
import highway_env

# # Agent
#from stable_baselines3 import DQN
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

import sys
from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# !apt-get install -y xvfb ffmpeg

In [16]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


In [11]:
from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np
import joblib
from tqdm import trange

llm stuff


In [4]:
!pip install -q transformers huggingface_hub[cli] bitsandbytes accelerate boto3

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [10]:
import boto3
import os
# Define Boto3 client for Bedrock
client = boto3.client("bedrock-runtime", region_name="us-east-1")
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Claude API settings
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
CLAUDE_API_URL = 'https://api.anthropic.com/v1/complete'


def claude_action(prompt1, assist1, prompt2, model='claude-v1', max_tokens_to_sample=50, temperature=0.7):
    """
    Sends prompts to Claude.ai and retrieves the recommended action.
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {CLAUDE_API_KEY}',
    }

    full_prompt = f"{prompt1}\n\n{assist1}\n\n{prompt2}"
    
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": full_prompt}],
            }
        ],
    }
    request = json.dumps(native_request)
    try:
        response = client.invoke_model(modelId=model_id, body=request)
        model_response = json.loads(response["body"].read())
        response_json = model_response["content"][0]["text"]
        #action_text = response_json.get('completion', '').strip()
        if 'Final decision:' in response_json:
            action = response_json.split('Final decision:')[-1].strip().upper()
            return action
        else:
            print(f"Unexpected response format: {action_text}")
            return 'IDLE'
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Claude.ai: {e}")
        time.sleep(1)
        return 'IDLE'

PROMPT

In [ ]:
# # Part 1: Initial prompt introducing the scenario
#         prompt1 = 'You are claude, a large language model. You are now acting as a mature driving assistant, who can give accurate and correct advice for human drivers in complex urban driving scenarios. The information in the current scenario:\n\
#                     You, the \'ego\' car, are now driving on a highway. You have already driven for 0 seconds.\n\
#                     The decision made by the agent LAST time step was \'FASTER\' (accelerate the vehicle).'

#         # Part 2: Driving rules that must be followed
#         rules = "There are several rules you need to follow when you drive on a highway:\n\
#                 1. Try to keep a safe distance from the car in front of you.\n\
#                 2. DON’T change lanes frequently. If you want to change lanes, double-check the safety of vehicles in the target lane."
        
#         prompt1 += rules

#         # Part 3: Attention points for decision making
#         att_points = "Here are your attention points:\n\
#                         1. You must output a decision when you finish this task. Your final output decision must be unique and not ambiguous. For example, you cannot say \"I can either keep lane or accelerate at the current time\".\n\
#                         2. You need to always remember your current lane ID, your available actions, and available lanes before you make any decision.\n\
#                         3. Once you have a decision, you should check the safety with all the vehicles affected by your decision.\n\
#                         4. If you verify a decision is unsafe, you should start a new one and verify its safety again from scratch."
        
#         prompt1 += att_points

#         # Part 4: Request for additional scenario details
#         assist1 = 'Understood. Please provide the current scenario or conditions, such as traffic density, speed of surrounding vehicles, your current speed, and any other relevant information, so I can recommend the best action.'

#         # Part 5: Describing the current highway scenario
#         prompt2 = 'Here is the current scenario:\n\
#         There are four lanes on the highway: Lane-1 (leftmost), Lane-2, Lane-3, Lane-4 (rightmost).\n\n'

#         # Extract information from the observations
#         #x, y, vx, vy = obs_[:, 1], obs_[:, 2], obs_[:, 3], obs_[:, 4]

#         # Ego vehicle details
#         ego_x, ego_y = 1759.3193,	0	
#         ego_vx, ego_vy = 25, 0

#         # Other vehicles' relative positions
#         veh_x, veh_y = x[1:] - ego_x, y[1:] - ego_y
#         veh_vx, veh_vy = vx[1:], vy[1:]

#         # Determine lane information for ego and other vehicles
#         #lanes = y // 4 + 1
#         ego_lane = 0
#         veh_lanes = lanes[1:]

#         # Left and right lane availability based on the ego vehicle's current lane
#         if ego_lane == 1:
#             ego_left_lane = 'Left lane: Not available\n'
#             ego_right_lane = 'Right lane: Lane-' + str(1 + 1) + '\n'
#         elif ego_lane == 4:
#             ego_left_lane = 'Left lane: Lane-' + str(4 - 1) + '\n'
#             ego_right_lane = 'Right lane: Not available\n'
#         else:
#             ego_left_lane = 'Left lane: Lane-' + str(2 - 1) + '\n'
#             ego_right_lane = 'Right lane: Lane-' + str(3 + 1) + '\n'

#         # Append ego vehicle information to prompt2
#         prompt2 += 'Ego vehicle:\n\
#         \tCurrent lane: Lane-' + str(1) + '\n' + '\t' + ego_left_lane + '\t' + ego_right_lane + '\tCurrent speed: ' + str(ego_vx) + ' m/s\n\n'

#         # Lane information including vehicles ahead in each lane
#         lane_info = 'Lane info:\n'
#         for i in range(4):
#             inds = np.where(veh_lanes == i + 1)[0]
#             num_v = len(inds)
#             if num_v > 0:
#                 # Find the closest vehicle in the current lane
#                 val, ind = self.find_smallest_positive(veh_x[inds])
#                 true_ind = inds[ind]
#                 lane_info += '\tLane-' + str(i + 1) + ': There are ' + str(num_v) + ' vehicle(s) in this lane ahead of ego vehicle, closest being ' + str(veh_x[true_ind]) + ' m ahead traveling at ' + str(veh_vx[true_ind]) + ' m/s.\n'
#             else:
#                 lane_info += '\tLane-' + str(i + 1) + ': No other vehicle ahead of ego vehicle.\n'
        
#         # Append lane information to prompt2
#         prompt2 += lane_info

#         # Part 6: Adding additional attention points and the final decision instruction
#         safety_verification = '\nAttention points:\n\
#         \t1.Safety is the main priority, You can stay IDLE or even Go slower but in no circumstance you should collide with lead vehicle.\n\
#         \t2.You are not supposed to change lane frequently only when its neccessary to keep the vehicle safe. Before changing lane check safety like safe distance and speed fro other vehicles\n\
#         \t3. Safety is a priority, but do not forget efficiency.\n\
#         \t4. you should only make a decesion once you have verified safety with other vehicles otherwise make a new decesion and verify its safety from scratch\n \
#         \t5. Your suggested action has to be one from the five listed actions - IDLE, SLOWER, FASTER, LANE_LEFT, LANE_RIGHT.\n\
#         Your last action was ' + self.prev_action + '.Please recommend action for the current scenario only in this format and DONT propound anything else other than \'Final decision: <final decision>\'.\n'

#         # Append the attention information to prompt2
#         prompt2 += safety_verification

#         # Return the three prompts
#         return prompt1, assist1, prompt2

In [5]:
action_dict = {
    0: 'LANE_LEFT',
    1: 'IDLE',
    2: 'LANE_RIGHT',
    3: 'FASTER',
    4: 'SLOWER'
}

In [8]:
def rf_query(obs):
    ##load model
    rf_model = joblib.load("models/rf.pkl")
    #print(f"Model loaded")
    ##
    obs_flat = obs.flatten().reshape(1, -1)  # Flatten and reshape observation
    return rf_model.predict(obs_flat)[0]

test llm

In [11]:
import os
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import RecordVideo
from tqdm import trange  # For progress tracking
from IPython.display import display, Video

# Base setting
vehicleCount = 10

# Environment configuration
config = {
    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 1,
    "show_trajectories": True,
    "render_agent": True,}

# Create directory for video storage if it doesn't exist
video_dir = 'videos/test_safe_Claude'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

# Create directory for action predictions storage if it doesn't exist
predictions_dir = 'predictions/test_safe_Claude'
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)

# Create and configure environment
env = gym.make('highway-v0', render_mode='rgb_array', config=config)

# Wrap environment with video recording
env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)


# Run test episodes and record them
for episode in trange(3, desc='Test episodes'):
    # Prepare to log the actions taken in this episode
    actions_file_path = os.path.join(predictions_dir, f'episode_{episode}_actions.txt')
    
    with open(actions_file_path, 'w') as action_file:
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            #action = 4 if np.random.rand() < 0.5 else 3
            action = rf_query(obs)
            obs, reward, done, truncated, info = env.step(int(action))
            
            # Log the action taken
            action_file.write(f"Action: {action}\n")
env.close()


# Display the recorded videos
show_videos()


/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")
/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/test_safe_Claude folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  war

KeyboardInterrupt: 

In [16]:
class MyHighwayEnvllm(gym.Env):

    def __init__(self, vehicleCount=10):
        super(MyHighwayEnvllm, self).__init__()
        # base setting
        self.vehicleCount = vehicleCount
        self.prev_action  = 'FASTER'

        # environment setting
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("highway-v0", config = self.config)
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf,high=np.inf,shape=(10,5),dtype=np.float32
        )

    def find_smallest_positive(self, arr):
        smallest_positive = float('inf')
        index = -1

        for i, value in enumerate(arr):
            if 0 < value < smallest_positive:
                smallest_positive = value
                index = i

        return smallest_positive, index

    def step(self, action):

        # Step the wrapped environment and capture all returned values
        obs, dqn_reward, done, truncated, info = self.env.step(action)

        ##combine rewards randomly
        if np.random.rand() <= 1:

            llm_response = rf_query(obs)

            l_acts  = 0

            if llm_response is not None:
                l_acts += 1
                ##modify action accordingly
                llm_act = llm_response

            if l_acts == 1:
                if llm_act == action:
                    llm_reward = 1
                else:
                    llm_reward = 0
                comb_reward = 0.7*dqn_reward + 0.3*llm_reward
            else:
                comb_reward = dqn_reward

        else:
            comb_reward = dqn_reward

        self.prev_action = action_dict[action]

        Reward = 1 / (1 + np.exp(-comb_reward))

        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs  # Make sure to return the observation

In [15]:
from stable_baselines3.common.vec_env import DummyVecEnv
env_llm = MyHighwayEnvllm()

/home/prachit/.local/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


NO OF TOKENS


In [ ]:
import tiktoken

def count_tokens(text, model='claude-v1'):
    tokenizer = tiktoken.encoding_for_model(model)
    tokens = tokenizer.encode(text)
    return len(tokens)

# Example usage
input_tokens = count_tokens(full_prompt)
output_tokens = count_tokens(response_json["content"])
total_tokens = input_tokens + output_tokens

In [21]:
model = DQN('MlpPolicy', env_llm.env,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            exploration_fraction=0.7,
            verbose=1)

model.learn(int(2e4))
model.save('models/dqn_model_claude')
# Download the model to your local machine

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.5     |
|    ep_rew_mean      | 8.59     |
|    exploration_rate | 0.997    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 42       |
|    total_timesteps  | 50       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.12     |
|    ep_rew_mean      | 6.14     |
|    exploration_rate | 0.995    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1        |
|    time_elapsed     | 63       |
|    total_timesteps  | 73       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.67     |
|    ep_rew_mean      | 5.11   

In [17]:
from gymnasium.wrappers import RecordVideo
# base setting
vehicleCount = 10

# environment setting
config = {

    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 2,
    "show_trajectories": True,
    "render_agent": True,
}

modelDqnllm = DQN.load("models/prachit_model_claude")

env = gym.make('highway-v0', render_mode='rgb_array', config = config)
env = RecordVideo(env, video_folder="new_videos", episode_trigger=lambda e: True)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done, truncated = env.reset(), False, False
    while not (done or truncated):
        action, _ = modelDqnllm.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()

/home/prachit/.local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Test episodes: 100%|██████████| 3/3 [01:14<00:00, 24.82s/it]
